In [1]:
from transformers.models.gemma3_with_talker.configuration_gemma3_with_talker import (
    Gemma3WithTalkerConfig,
)
from transformers.models.gemma3_with_talker.modeling_gemma3_with_talker import (
    Gemma3WithTalkerForConditionalGeneration,
    Gemma3WithTalkerThinkerForCausalLM,
)
from transformers.models.gemma3_with_talker.configuration_gemma3_with_talker import *
from transformers import AutoConfig, AutoModel

from loguru import logger
from transformers import AutoConfig

In [4]:
# from transformers import AutoConfig, AutoModel

# from loguru import logger
# from transformers import AutoConfig

from transformers.models.gemma3_with_talker.configuration_gemma3_with_talker import (
    Gemma3WithTalkerThinkerTextConfig,
)
from transformers.models.gemma3_with_talker.modeling_gemma3_with_talker import (
    Gemma3WithTalkerThinkerTextModel,
)

configs = {
    # "gemma3_thinker": Gemma3ThinkerConfig,
    # "gemma3_talker": Gemma3TalkerConfig,
    # "gemma3_token2wav": Gemma3Token2WavConfig,
    # "gemma3_with_talker": Gemma3WithTalkerConfig,
    "gemma3_with_talker_thinker_text": Gemma3WithTalkerThinkerTextConfig,
    # "gemma3_bigvgan": Gemma3BigVGANConfig,
    # "gemma3_dit": Gemma3DiTConfig,
}
for name, conf in configs.items():
    AutoConfig.register(name, conf)

# AutoConfig.register("gemma3_with_talker", Gemma3WithTalkerConfig)
AutoModel.register(Gemma3WithTalkerThinkerTextConfig, Gemma3WithTalkerThinkerTextModel)

In [2]:
import os

os.environ["LIBRARY_PATH"] = "/usr/local/cuda/lib64/stubs:$LIBRARY_PATH"

In [5]:
model = Gemma3WithTalkerForConditionalGeneration.from_pretrained(
    # "/data1/nowshad/models/gemma3_with_talker_base",
    "/data1/nowshad/models/gemma3_with_talker_merge",
    device_map="cuda:6",
    # config=config,
    torch_dtype="bfloat16",
    # ignore_mismatched_sizes=True,
)

2025-06-02 11:39:02.128 | INFO     | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4827 - Inside from pretrained
2025-06-02 11:39:02.129 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4828 - Config None
2025-06-02 11:39:02.129 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4829 - Model args
2025-06-02 11:39:02.129 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4831 - kwargs


Inside tie weights
Inside tie weights getattr tie_word_embeddings


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Inside tie weights
Inside tie weights getattr tie_word_embeddings


In [10]:
tensors["thinker.model.lm_head.weight"].shape

torch.Size([262145, 3840])

In [ ]:
import torch.nn as nn

lm_head = nn.Linear(3840, 262145, bias=False)
lm_head.weight = nn.Parameter(tensors["thinker.model.lm_head.weight"])
lm_head

Linear(in_features=3840, out_features=262145, bias=False)

In [21]:
nn.Parameter(tensors["thinker.model.lm_head.weight"]).values

<function Parameter.values>

In [ ]:
thinker_model.lm_head.weight.to("cuda:6")

tensor([[-5.0964e-03, -3.8910e-04, -3.7994e-03,  ...,  1.8234e-03,
          1.5259e-03, -5.0659e-03],
        [ 4.4556e-03,  1.4282e-02,  5.6458e-04,  ..., -1.3428e-02,
          8.2016e-04,  9.6436e-03],
        [ 2.7222e-02, -1.1902e-03, -2.0020e-02,  ...,  8.5449e-03,
          1.6113e-02,  7.3242e-04],
        ...,
        [-3.7384e-03, -2.0447e-03, -3.6316e-03,  ...,  1.9531e-03,
          8.9645e-04, -4.6692e-03],
        [-2.3041e-03, -1.6174e-03, -6.2866e-03,  ...,  2.7008e-03,
          2.9755e-03, -5.1270e-03],
        [-7.9155e-05, -8.5831e-05, -6.1512e-05,  ..., -7.3910e-05,
          1.2040e-05,  2.9802e-06]], device='cuda:6', dtype=torch.bfloat16,
       grad_fn=<ToCopyBackward0>)

In [13]:
x = thinker_model.lm_head.weight.to("cuda:6")
x

tensor([[-5.0964e-03, -3.8910e-04, -3.7994e-03,  ...,  1.8234e-03,
          1.5259e-03, -5.0659e-03],
        [ 4.4556e-03,  1.4282e-02,  5.6458e-04,  ..., -1.3428e-02,
          8.2016e-04,  9.6436e-03],
        [ 2.7222e-02, -1.1902e-03, -2.0020e-02,  ...,  8.5449e-03,
          1.6113e-02,  7.3242e-04],
        ...,
        [-3.7384e-03, -2.0447e-03, -3.6316e-03,  ...,  1.9531e-03,
          8.9645e-04, -4.6692e-03],
        [-2.3041e-03, -1.6174e-03, -6.2866e-03,  ...,  2.7008e-03,
          2.9755e-03, -5.1270e-03],
        [-7.9155e-05, -8.5831e-05, -6.1512e-05,  ..., -7.3910e-05,
          1.2040e-05,  2.9802e-06]], device='cuda:6', dtype=torch.bfloat16,
       grad_fn=<ToCopyBackward0>)

In [6]:
model.thinker.lm_head.weight = thinker_model.lm_head.weight
# model.thinker.lm_head.weight = x

In [5]:
# thinker_model = Gemma3WithTalkerThinkerForCausalLM.from_pretrained(
#     "/data1/nowshad/models/gemma3_with_talker_base",
#     # "/data1/nowshad/models/gemma3_with_talker_merge",
#     device_map="cuda:6",
#     # config=config,
#     torch_dtype="bfloat16",
#     # ignore_mismatched_sizes=True,
# )

In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForImageTextToText,
)

thinker_model = AutoModelForImageTextToText.from_pretrained(
    "/data1/nowshad/models/gemma3_12b_ft_20250519",
    # "/data1/nowshad/models/Qwen2-7B",
    # "/data1/nowshad/models/Qwen2.5-VL-7B-Instruct_processor",
    torch_dtype="bfloat16",
    device_map="cuda:6",
    # trust_remote_code=True,
)

Inside tie weights
Inside tie weights getattr tie_word_embeddings


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Inside tie weights
Inside tie weights getattr tie_word_embeddings


In [24]:
thinker_model.lm_head

Linear(in_features=3840, out_features=262145, bias=False)

In [7]:
model.thinker = thinker_model

In [1]:
262208 - 262145

63

In [6]:
from transformers import AutoTokenizer

thinker_tokenizer = AutoTokenizer.from_pretrained(
    "/data1/nowshad/models/gemma3_12b_ft_20250519"
)
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is your name?"},
        ],
    },
]
text = thinker_tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# text = "What is your name?"
inputs = thinker_tokenizer([text], return_tensors="pt").to("cuda:6")

In [8]:
thinker_result = model.generate(**inputs)

Setting `pad_token_id` to `eos_token_id`:8292 for open-end generation.


In [7]:
# thinker_result = model(**inputs)

TypeError: _forward_unimplemented() got an unexpected keyword argument 'input_ids'

In [9]:
text_ids, audio = thinker_result

In [10]:
thinker_tokenizer.batch_decode(text_ids, skip_special_tokens=True)[0]

"user\nYou are a helpful assistant.\n\nWhat is your name?\nmodel\nMy name is Gemma! I'm a large language model created by the Gemma team at Google DeepMind. It's nice to meet you. 😊"

In [9]:
import soundfile as sf

sf.write(
    "output.wav",
    audio.reshape(-1).detach().cpu().numpy(),
    samplerate=24000,
)

In [13]:
model.save_pretrained("/data1/nowshad/models/gemma3_with_talker_merge")

In [8]:
from safetensors import safe_open


def convert_tensor(file_path):
    tensors = {}
    with safe_open(file_path, framework="pt", device=6) as f:
        for k in f.keys():
            if "language_model.model" in k:
                new_k = k.replace(
                    "language_model.model", "thinker.model.language_model"
                )
            elif "multi_modal_projector" in k:
                new_k = "thinker.model." + k
            elif "vision_tower" in k:
                new_k = "thinker.model." + k
            else:
                new_k = k
            tensors[new_k] = f.get_tensor(k)
    return tensors

In [ ]:
from safetensors.torch import save_file
import os
from tqdm import tqdm

read_path = "/data1/nowshad/models/gemma3_with_talker_base"
save_path = "/data1/nowshad/models/gemma3_with_talker_base/mdf"
files = os.listdir(read_path)
for file in tqdm(files):
    if file.endswith("safetensors"):
        print(f"Working on {file}")
        new_tensor = convert_tensor(os.path.join(read_path, file))
        save_file(new_tensor, os.path.join(save_path, file))
    else:
        continue

  0%|          | 0/12 [00:00<?, ?it/s]

Working on model-00001-of-00005.safetensors


  8%|▊         | 1/12 [00:08<01:30,  8.20s/it]

Working on model-00002-of-00005.safetensors


 33%|███▎      | 4/12 [00:15<00:29,  3.63s/it]

...

 58%|█████▊    | 7/12 [00:33<00:23,  4.61s/it]

Working on model-00005-of-00005.safetensors


 83%|████████▎ | 10/12 [00:39<00:06,  3.26s/it]

Working on model-00003-of-00005.safetensors


100%|██████████| 12/12 [00:46<00:00,  3.88s/it]


In [7]:
x = "something"
"some" in x

True

In [7]:
lm_head_file_path = (
    "/data1/nowshad/models/DeepSeek-R1-0528-Qwen3-8B/model-00002-of-000002.safetensors"
)
embed_file_path = (
    "/data1/nowshad/models/DeepSeek-R1-0528-Qwen3-8B/model-00001-of-000002.safetensors"
)
from safetensors import safe_open

with safe_open(embed_file_path, framework="pt", device=6) as f:
    for k in f.keys():
        if "embed_tokens" in k:
            print(f"Embed token weights shape {k} is {f.get_tensor(k).shape}")

with safe_open(lm_head_file_path, framework="pt", device=6) as f:
    for k in f.keys():
        if "lm_head" in k:
            print(f"LM head weights shape {k} is {f.get_tensor(k).shape}")

Embed token weights shape model.embed_tokens.weight is torch.Size([151936, 4096])
LM head weights shape lm_head.weight is torch.Size([151936, 4096])


In [3]:
x = "something is {age} and country is {country} and else is {style}"
x.format(age="25-30", country="SG", style="Playful")

'something is 25-30 and country is SG and else is Playful'

In [ ]:
"thinker.model.language_model.embed_tokens.weight": "model-00001-of-00005.safetensors"

"lm_head.weight"

In [5]:
from safetensors import safe_open

tensors = {}
with safe_open(
    "/data1/nowshad/models/gemma3_with_talker_base/model-00001-of-00005.safetensors",
    framework="pt",
    device=6,
) as f:
    for k in f.keys():
        if "embed_tokens" in k:
            print(f"Embed token weights shape {k} is {f.get_tensor(k).shape}")
            tensors["thinker.model.lm_head.weight"] = f.get_tensor(k)

Embed token weights shape thinker.model.language_model.embed_tokens.weight is torch.Size([262145, 3840])


In [7]:
tensors["thinker.model.lm_head.weight"]

tensor([[-5.0964e-03, -3.8910e-04, -3.7994e-03,  ...,  1.8234e-03,
          1.5259e-03, -5.0659e-03],
        [ 4.4556e-03,  1.4282e-02,  5.6458e-04,  ..., -1.3428e-02,
          8.2016e-04,  9.6436e-03],
        [ 2.7222e-02, -1.1902e-03, -2.0020e-02,  ...,  8.5449e-03,
          1.6113e-02,  7.3242e-04],
        ...,
        [-3.7384e-03, -2.0447e-03, -3.6316e-03,  ...,  1.9531e-03,
          8.9645e-04, -4.6692e-03],
        [-2.3041e-03, -1.6174e-03, -6.2866e-03,  ...,  2.7008e-03,
          2.9755e-03, -5.1270e-03],
        [-7.9155e-05, -8.5831e-05, -6.1512e-05,  ..., -7.3910e-05,
          1.2040e-05,  2.9802e-06]], device='cuda:6', dtype=torch.bfloat16)

In [17]:
from safetensors import safe_open

tensors = {}
with safe_open(
    "/data1/nowshad/models/gemma3_with_talker_base/model-00005-of-00005.safetensors",
    framework="pt",
    device=6,
) as f:
    for k in f.keys():
        # tensors[k] = f.get_tensor(k)
        # if "lm_head" in k:
        #     print(f"Embed token weights shape {k} is {f.get_tensor(k).shape}")
        print(f"{k} shape is {f.get_tensor(k).shape}")
        # tensors[k] = f.get_tensor(k)

thinker.model.language_model.layers.37.input_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.37.mlp.down_proj.weight shape is torch.Size([3840, 15360])
thinker.model.language_model.layers.37.post_attention_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.37.post_feedforward_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.37.pre_feedforward_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.38.input_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.38.mlp.down_proj.weight shape is torch.Size([3840, 15360])
thinker.model.language_model.layers.38.mlp.gate_proj.weight shape is torch.Size([15360, 3840])
thinker.model.language_model.layers.38.mlp.up_proj.weight shape is torch.Size([15360, 3840])
thinker.model.language_model.layers.38.post_attention_layernorm.weight shape is torch.Size([3840])
thinker.model.language_model.layers.38.post

In [ ]:
del tensors["thinker.model.lm_head.weight"]

In [12]:
from safetensors.torch import save_file

save_file(
    tensors,
    "/data1/nowshad/models/gemma3_with_talker_base/model-00005-of-00005.safetensors",
)